In [16]:
# Data Preparation and Analysis for Amazon Sales Data

## Data Exploration
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [9]:
# Load the CSV file
file_path = 'AmazonDataSales.csv'  # Update this path
amazon_data = pd.read_csv(file_path)

# Display basic information about the data
amazon_data.info()
amazon_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   index               128975 non-null  int64  
 1   Order ID            128975 non-null  object 
 2   Date                128975 non-null  object 
 3   Status              128975 non-null  object 
 4   Fulfilment          128975 non-null  object 
 5   Sales Channel       128975 non-null  object 
 6   ship-service-level  128975 non-null  object 
 7   Style               128975 non-null  object 
 8   SKU                 128975 non-null  object 
 9   Category            128975 non-null  object 
 10  Size                128975 non-null  object 
 11  ASIN                128975 non-null  object 
 12  Courier Status      122103 non-null  object 
 13  Qty                 128975 non-null  int64  
 14  currency            121180 non-null  object 
 15  Amount              121180 non-nul

C:\Users\mehrn\AppData\Local\Temp\ipykernel_42280\3892149674.py:3: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  amazon_data = pd.read_csv(file_path)


,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,...,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,...,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,NaN,False,Easy Ship,NaN
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,...,INR,574.00,CHENNAI,TAMIL NADU,600073.0,IN,NaN,False,NaN,NaN


In [10]:
# Data Cleaning and Preprocessing

# a. Handle missing values and inconsistencies
# Dropping columns with a high percentage of missing values (e.g., 'Unnamed: 22', 'fulfilled-by')
amazon_data_cleaned = amazon_data.drop(['Unnamed: 22', 'fulfilled-by'], axis=1)

# Filling missing values in other columns with 'Unknown' or the most common value
amazon_data_cleaned['Courier Status'].fillna('Unknown', inplace=True)
amazon_data_cleaned['currency'].fillna(amazon_data_cleaned['currency'].mode()[0], inplace=True)
amazon_data_cleaned['Amount'].fillna(amazon_data_cleaned['Amount'].mean(), inplace=True)
amazon_data_cleaned['ship-city'].fillna('Unknown', inplace=True)
amazon_data_cleaned['ship-state'].fillna('Unknown', inplace=True)
amazon_data_cleaned['ship-postal-code'].fillna('Unknown', inplace=True)
amazon_data_cleaned['ship-country'].fillna('Unknown', inplace=True)
amazon_data_cleaned['promotion-ids'].fillna('None', inplace=True)

# b. Standardize text data for consistency
text_columns = ['Category', 'Size', 'Style']
for col in text_columns:
    amazon_data_cleaned[col] = amazon_data_cleaned[col].str.lower().str.strip()

# c. Convert 'Date' to an appropriate datetime format
amazon_data_cleaned['Date'] = pd.to_datetime(amazon_data_cleaned['Date'], format='%m-%d-%y', errors='coerce')

# Display the cleaned data
amazon_data_cleaned.head()

C:\Users\mehrn\AppData\Local\Temp\ipykernel_42280\3449316141.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  amazon_data_cleaned['ship-postal-code'].fillna('Unknown', inplace=True)


,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,Courier Status,Qty,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B
0,0,405-8078784-5731545,2022-04-30,Cancelled,Merchant,Amazon.in,Standard,set389,SET389-KR-NP-S,set,...,Unknown,0,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,None,False
1,1,171-9198151-1101146,2022-04-30,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,jne3781,JNE3781-KR-XXXL,kurta,...,Shipped,1,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False
2,2,404-0687676-7273146,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,jne3371,JNE3371-KR-XL,kurta,...,Shipped,1,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True
3,3,403-9615377-8133951,2022-04-30,Cancelled,Merchant,Amazon.in,Standard,j0341,J0341-DR-L,western dress,...,Unknown,0,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,None,False
4,4,407-1069790-7240320,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,jne3671,JNE3671-TU-XXXL,top,...,Shipped,1,INR,574.00,CHENNAI,TAMIL NADU,600073.0,IN,None,False


In [12]:
# Save the cleaned data to a new CSV file
cleaned_file_path = 'Cleaned_AmazonDataSales.csv'  # Update this path
amazon_data_cleaned.to_csv(cleaned_file_path, index=False)

In [13]:
# Deriving new features from the data

# a. Derive 'Month' and 'Year' from 'Date'
amazon_data_cleaned['Month'] = amazon_data_cleaned['Date'].dt.month
amazon_data_cleaned['Year'] = amazon_data_cleaned['Date'].dt.year

# b. Create binary features from 'B2B'
# Assuming 'B2B' is already a boolean column, we can convert it to binary (0 and 1)
amazon_data_cleaned['B2B_binary'] = amazon_data_cleaned['B2B'].astype(int)

# c. Group 'Amount' into different sales intervals for categorization
# Define the intervals
intervals = [0, 200, 500, 1000, 5000, float('inf')]
labels = ['0-200', '201-500', '501-1000', '1001-5000', '5000+']

# Create a new categorical column
amazon_data_cleaned['Sales_Range'] = pd.cut(amazon_data_cleaned['Amount'], bins=intervals, labels=labels)

# Display the data with new features
amazon_data_cleaned.head()

,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,Month,Year,B2B_binary,Sales_Range
0,0,405-8078784-5731545,2022-04-30,Cancelled,Merchant,Amazon.in,Standard,set389,SET389-KR-NP-S,set,...,MUMBAI,MAHARASHTRA,400081.0,IN,None,False,4,2022,0,501-1000
1,1,171-9198151-1101146,2022-04-30,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,jne3781,JNE3781-KR-XXXL,kurta,...,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,4,2022,0,201-500
2,2,404-0687676-7273146,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,jne3371,JNE3371-KR-XL,kurta,...,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,4,2022,1,201-500
3,3,403-9615377-8133951,2022-04-30,Cancelled,Merchant,Amazon.in,Standard,j0341,J0341-DR-L,western dress,...,PUDUCHERRY,PUDUCHERRY,605008.0,IN,None,False,4,2022,0,501-1000
4,4,407-1069790-7240320,2022-04-30,Shipped,Amazon,Amazon.in,Expedited,jne3671,JNE3671-TU-XXXL,top,...,CHENNAI,TAMIL NADU,600073.0,IN,None,False,4,2022,0,501-1000


In [17]:
# Normalizing/standardizing numerical values ('Qty' and 'Amount')
numerical_features = ['Qty', 'Amount']
scaler = StandardScaler()
amazon_data[numerical_features] = scaler.fit_transform(amazon_data[numerical_features])

# Using LabelEncoder for categorical variables
label_encoders = {}
for col in ['Category', 'Style', 'Size', 'Sales Channel ']:
    le = LabelEncoder()
    amazon_data[col] = le.fit_transform(amazon_data[col].astype(str))
    label_encoders[col] = le  # Store the label encoder for each column

# Display the data with label encoded features
amazon_data.head()


,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,0,Standard,1342,SET389-KR-NP-S,5,...,INR,-0.003348,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,0,Standard,847,JNE3781-KR-XXXL,8,...,INR,-0.862562,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,0,Expedited,533,JNE3371-KR-XL,8,...,INR,-1.136378,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,0,Standard,373,J0341-DR-L,7,...,INR,0.372563,PUDUCHERRY,PUDUCHERRY,605008.0,IN,NaN,False,Easy Ship,NaN
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon,0,Expedited,754,JNE3671-TU-XXXL,6,...,INR,-0.265145,CHENNAI,TAMIL NADU,600073.0,IN,NaN,False,NaN,NaN
